# 不分词，字向量法训练分类

@PM


### 准备工作： 连接数据库并获取各类文本数据到列表


In [35]:
import sqlite3
conn = sqlite3.connect("../data/database/texts.db")
print ('**Opened database successfully**\n')
c = conn.cursor()
ci_data=[]
cursor = c.execute("SELECT * from ci order by id asc limit 5000")
#cursor = c.execute("SELECT * from ci order by id")
for row in cursor:
    ci_data.append(row)
print("读取的词数据样例：")
print(ci_data[-2:])

poet_data = []
cursor=c.execute("SELECT * FROM poet order by id asc limit 5000")
for row in cursor:
    poet_data.append(row)
print("\n读取的诗数据样例：")
print(poet_data[-2:])

classical_data = []
cursor=c.execute("SELECT * FROM classical order by id asc limit 5000")
for row in cursor:
    classical_data.append(row)
print("\n读取的文言文数据样例：")
print(classical_data[-2:])

journal_data = []
cursor=c.execute("SELECT * FROM journal order by id asc limit 5000")
for row in cursor:
    journal_data.append(row)
print("\n读取的期刊数据样例：")
print(journal_data[:2])
    
news_data = []
cursor=c.execute("SELECT * FROM news order by id asc limit 5000")
for row in cursor:
    news_data.append(row)
print("\n读取的新闻数据样例：")
print(news_data[:2])

**Opened database successfully**

读取的词数据样例：
[(50004999, '玉带云袍童顶露。 一生笑傲知何故。 万里归来方旦暮。 休疑虑。 大千捏在豪端聚。 不解犁田分亩步。 却能对客鸣花鼓。 忽共老安相耳语。 还推去。 莫来拦我球门路。', 'ci', 4999), (50005000, '不怕石头行路滑。 归来那爱驹儿踏。 言下百骸俱拨撒。 无剩法。 灵然昼夜光通达。 古寺天寒还恶发。 夜将木佛齐烧杀。 炙背横眠真快活。 憨抹挞。 从教院主无须发。', 'ci', 5000)]

读取的诗数据样例：
[(40004999, '溪头买船归鄂渚，一櫂清风夜蟾午。 行尽韶山半月程，白云儿就青山父。 岩头老子末后句，此到将知问不语。 无缝塔前风色高，洞然不隔丝毫许。 丁宁此去快须回，吾家之子要全才。 不坐两头明暗路，偏中归去正中来。 阿呵呵，胸次风流知几何。 ', 'poet', 'fcca8065-2a8a-4560-b575-ef26e1b16bd6'), (40005000, '木落山骨瘦，水退沙痕隆。 渔舟弄江日，一叶浮软红。 眼底事不俗，道人心若冲。 旛竿头进步，云水展家风。 ', 'poet', 'bbf6440f-763a-463e-a063-874db1646733')]

读取的文言文数据样例：
[(10005000, '\u3000\u3000（二）段之錍，或说为斧，或说为箭，据文重五斤观之，似不是箭，孙谓“击”之形为三隅，无据。\n', 'classical', 'c_list_100.txt'), (10005001, '\u3000\u3000２２诸林木渥水中，无过一茷。\n', 'classical', 'c_list_100.txt')]

读取的期刊数据样例：
[(20000001, '作者程苏东, 北京大学中文系讲师 (北京100871) 。\n', 'modern', 'j_list_1.txt'), (20000002, '“失控的文本”这一概念是我在研究《汉书·五行志》体例问题时产生的想法。班固以刘向《洪范五行传论》为基础, 试图纂集董仲舒《春秋》灾异说、许商《五行传记》、刘歆《洪范五行传论》等不同系统的灾异学论著, 整合成具有集大成性质的西汉灾异学总论, 从文本生成的角度而言, 《汉书·五行

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
classifier = MultinomialNB()
tv = TfidfVectorizer(analyzer='char',max_features=5000, lowercase = False)


def train_NBayes(x_train, x_test, y_train, y_test):
    #模型训练
    classifier.fit(vectorizer.transform(x_train), y_train)
    #使用训练好的模型进行预测
    print("朴素贝叶斯得分："+str(classifier.score(vectorizer.transform(x_test), y_test)))
    
def train_TfIdf(x_train, x_test, y_train, y_test):
    tv.fit(x_train)
    #模型训练
    classifier.fit(tv.transform(x_train), y_train)
    #利用训练好的模型测试
    print("TF-IDF得分："+str(classifier.score(tv.transform(x_test), y_test)))
def train_nGram(x_train, x_test, y_train, y_test,n):
    #转向量
    tv_2gram = TfidfVectorizer(analyzer='char', max_features=5000, ngram_range=(1,n),lowercase = False)
    tv_2gram.fit(x_train)
    #训练模型
    clf_2gram = MultinomialNB()
    clf_2gram.fit(tv_2gram.transform(x_train), y_train)
    #预测
    print("ngram,n为"+str(n)+"时得分："+str(clf_2gram.score(tv_2gram.transform(x_test), y_test)))

## ⅠBasic方案：对原文本拆分成单字
### 1. 暴力拆字


In [37]:
data = ci_data + poet_data + classical_data + journal_data + news_data
data_list = []
for line in data:
    data_list.append(list(line))

print(data_list[6000])
for item in data_list:
    item[1] = ' '.join(list(item[1]))
print(data_list[6000])
print(len(data_list))

[40001001, '当涂当涂见，芜湖芜湖见。 八月十五夜，一似没柄扇。 ', 'poet', 'f0890360-c046-4c63-8121-76d8fe8d6195']
[40001001, '当 涂 当 涂 见 ， 芜 湖 芜 湖 见 。   八 月 十 五 夜 ， 一 似 没 柄 扇 。  ', 'poet', 'f0890360-c046-4c63-8121-76d8fe8d6195']
25000


### 2. 建立训练集、测试集

In [38]:
train_data=[]
train_target=[]
for i in range(0,len(data)):
    train_data.append(data[i][1])
    train_target.append(data[i][2])
print(train_data[:3])
print(train_target[15000:15003])

['气和玉烛，睿化著鸿明。 缇管一阳生。 郊盛礼燔柴毕，旋轸凤凰城。 森罗仪卫振华缨。 载路溢欢声。 皇图大业超前古，垂象泰阶平。 岁时丰衍，九土乐升平。 睹寰海澄清。 道高尧舜垂衣治，日月并文明。 嘉禾甘露登歌荐，云物焕祥经。 兢兢惕惕持谦德，未许禅云亭。', '严夜警，铜莲漏迟迟。 清禁肃，森陛戟，羽卫俨皇闱。 角声励，钲鼓攸宜。 金管成雅奏，逐吹逶迤。 荐苍璧，郊祀神祗。 属景运纯禧。 京坻丰衍，群材乐育，诸侯述职，盛德服蛮夷。 殊祥萃，九苞丹凤来仪。 膏露降，和气洽，三秀焕灵芝。 鸿猷播，史册相辉。 张四维。 卜世永固丕基。 敷玄化，荡荡无为。 合尧舜文思。 混并寰宇，休牛归马，销金偃革，蹈咏庆昌期。', '承宝运，驯致隆平。 鸿庆被寰瀛。 时清俗阜，治定功成。 遐迩咏由庚。 严郊祀，文物声明。 会天正、星拱奏严更。 布羽仪簪缨。 宸心虔洁，明德播惟馨。 动苍冥。 神降享精诚。 燔柴半，万乘移天仗，肃銮辂旋衡。 千官云拥，群后葵倾。 玉帛旅明庭。 韶荐，金奏谐声。 集休亨。 皇泽浃黎庶，普率洽恩荣。 仰钦元后，睿圣贯三灵。 万邦宁。 景贶福千龄。']
['modern', 'modern', 'modern']


In [39]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.3,random_state=0)

train_data=[]
train_target=[]
for i in range(0,25000):
    train_data.append(data_list[i][1])
    train_target.append(data_list[i][2])

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_target,test_size=0.4,random_state=0)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector = cv.fit_transform(x_train)
vector.todense()

matrix([[0, 1, 7, ..., 0, 0, 0],
        [0, 0, 2, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 7, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 0, 6, ..., 0, 0, 0]], dtype=int64)

In [41]:
cv.vocabulary_

{'以': 571,
 '此': 3555,
 '为': 474,
 '主': 475,
 '旨': 3044,
 ',': 14,
 ' ': 2,
 '他': 563,
 '将': 1955,
 '中': 466,
 '国': 1490,
 '数': 2985,
 '千': 1047,
 '年': 2205,
 '学': 1866,
 '术': 3181,
 '思': 2412,
 '想': 2524,
 '史': 1147,
 '分': 904,
 '胚': 5637,
 '胎': 5633,
 '、': 276,
 '全': 822,
 '盛': 4652,
 '儒': 791,
 '统': 5365,
 '一': 436,
 '老': 5536,
 '佛': 628,
 '混': 3878,
 '合': 1161,
 '和': 1239,
 '衰': 6368,
 '落': 6037,
 '七': 438,
 '个': 464,
 '时': 3052,
 '代': 569,
 '并': 2206,
 '其': 833,
 '所': 2636,
 '处': 1659,
 '的': 4616,
 '当': 2307,
 '复': 1661,
 '兴': 831,
 '也': 499,
 '就': 1979,
 '是': 3078,
 '第': 5064,
 '八': 824,
 '。': 277,
 '从': 558,
 '后': 1166,
 '来': 3208,
 '梁': 3350,
 '启': 1191,
 '超': 6837,
 '改': 2959,
 '题': 7853,
 '“': 146,
 '近': 7052,
 '世': 451,
 '”': 147,
 '看': 4672,
 '似': 612,
 '乎': 487,
 '已': 2136,
 '经': 5351,
 '不': 444,
 '这': 7058,
 '期': 3174,
 '只': 1139,
 '认': 6576,
 '有': 3163,
 '清': 3884,
 '之': 484,
 '大': 1673,
 '抵': 2699,
 '述': 7074,
 '而': 5543,
 '无': 3039,
 '作': 629,
 '故': 2964,
 '可': 1144,
 

In [42]:
vectorizer = CountVectorizer(analyzer='char', max_features=5000)
vectorizer.fit(x_train)
print(x_train[:3])

['以此为主旨, 他将中国数千年学术思想史分为胚胎、全盛、儒学统一、老学、佛学、儒佛混合和衰落七个时代, 并以其所处的当时为复兴时代, 也就是第八时代。从后来梁启超将第七时代改题为“近世学术”看, 他似乎已经不以这一时期的中国学术为衰落。只是认为“有清一代之学术, 大抵述而无作, 学而不思, 故可谓之为思想最衰时代”。\n', '今年六十七，老病随缘且遣日。 今年记却来年事，来年记著今朝日。 ', '近日，网友一条眼保健操“无用论”微博使得眼保健操话题再度成为关注焦点，该微博称“眼保健操残害中国青少年４９年了，许多学生用脏手按摩导致红眼病、眼部感染等。”专家则表示，网友的说法是用常识代替了科学，过于绝对。（７月１７日《扬子晚报》）Ｗ魑近视高发国度，多年来，我国青少年的近视率始终位居世界前列，稳升不降，有人就把怀疑的目光投向“那些年，我们一起做过的眼保健操”。作为生活学习中的一项重要记忆，眼保健操曾伴随了几代人的成长，可以说是已经深深打上了时代与集体生活的印记。但同时，由于缺乏专业论证及临床实践，这项“全民运动”也屡遭质疑：在此次网友的“眼保健操无用论”抛出之前，方舟子曾撰文质疑眼保健操的效果；时间再往前推，“文革期间”学校恢复正常上课之后，眼保健操也二度复出，彼时，也有人质疑这套操是否可行，是否得到了专家的论证。Ｗ魑与穴位按摩、经络相关的一套保健操，在穴位、经络之说本身就尚未经过科学论证的情境下，要用确凿的证据与实验，证明它的可行性，可能目前并不现实。这就像甘肃省曾组织医务人员集体“打通任督二脉”，却惹人讥讽一样。从自由选择的角度来说，人们反感的可能并不是这套眼保健操不科学无论证，而是这样一套纯属眼疾者自创的保健操，怎么就稀里糊涂成了一套普及多年的“全民运动”？？凸鄣亟玻青少年之所以近视，恐怕也不能把责任简单归咎于一个尚无法论证的眼保健操。应试教育压力下，大多数孩子把大量的时间都耗费在书本学习上，眼睛与神经都长期处于高度紧张状态，休息时间，大多数孩子又将眼睛盯住了网络和电视，户外活动与“望远”时间减少。另外，不得不提的是，一些商家打着慈善旗号的“爱眼工程”，用廉价的镜片吸引学生购买，甚至为推销产品，传输错误的用眼常识……种种因素作用下，视力下降就成为必然。Ｔ谝桓鱿肮呓邮苋ㄍ灌输的时代，质疑一些已经形成生活方式，大家不敢轻易质疑的东西，其实是好事。但倘若要触及问题根本

In [43]:
print("暴力拆字方法的训练结果：")
train_NBayes(x_train, x_test, y_train, y_test)
train_TfIdf(x_train, x_test, y_train, y_test)
train_nGram(x_train, x_test, y_train, y_test,2)

暴力拆字方法的训练结果：
朴素贝叶斯得分：0.9402666666666667
TF-IDF得分：0.9313333333333333
ngram,n为2时得分：0.9466666666666667


## Pro: 清理时去掉符号（利用正则表达式）

In [24]:
import re

train_data_pro = []
for paragraph in train_data:
    train_data_pro.append(' '.join(list((re.sub("[\s+\.\!\/_,$%^*)(+\"\']+|[+——！，。？、~@#￥%……&*（）]", "",paragraph)))))


print("去除符号成功：\n样例：【"+train_data[6000]+"】=========>【"+train_data_pro[6000]+"】")

去除符号成功：
样例：【当涂当涂见，芜湖芜湖见。 八月十五夜，一似没柄扇。 】=========>【当 涂 当 涂 见 芜 湖 芜 湖 见 八 月 十 五 夜 一 似 没 柄 扇】


In [25]:
x_train_pro, x_test_pro, y_train, y_test = train_test_split(train_data_pro, train_target,test_size=0.4,random_state=0)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector_pro = cv.fit_transform(x_train_pro)
vector_pro.todense()
vectorizer_pro = CountVectorizer(analyzer='char', max_features=5000)
vectorizer_pro.fit(x_train_pro)
print(x_train_pro[:3])

['尊 前 一 把 横 波 溜 彼 此 心 儿 有 曲 屏 深 幌 解 香 罗 花 灯 微 透 偎 人 欲 语 眉 先 皱 红 玉 困 春 酒 为 问 鸳 衾 这 回 後 几 时 重 又', '本 报 讯 记 者 刘 磊 涛 克 莱 斯 勒 耗 费 １ ０ 亿 美 金 打 造 的 全 新 ３ ０ ０ Ｃ 将 于 本 月 底 正 式 上 市 目 前 针 对 这 款 后 驱 豪 华 轿 车 克 莱 斯 勒 在 华 ４ Ｓ 店 已 开 始 全 面 接 受 客 户 预 订 新 车 定 价 预 计 将 低 于 ５ ０ 万 元 Ｍ 夤 鄯 矫 妫 新 ３ ０ ０ Ｃ 延 续 了 老 款 直 线 条 硬 朗 的 特 质 整 车 尺 寸 略 有 变 大 长 宽 高 为 ５ ０ ６ ６ × １ ９ ０ ２ × １ ４ ８ ８ ｍ ｍ 而 轴 距 为 ３ ０ ５ ２ ｍ ｍ 变 化 最 大 的 在 前 脸 上 全 新 的 镀 铬 格 栅 凌 厉 霸 气 由 于 新 ３ ０ ０ Ｃ 是 克 莱 斯 勒 并 入 菲 亚 特 集 团 后 的 第 一 款 车 型 采 用 了 新 的 克 莱 斯 勒 车 标 飞 翼 标 志 更 加 漂 亮 美 观 Ｄ 谑 畏 矫 妫 全 新 ３ ０ ０ Ｃ 一 改 以 往 粗 犷 的 做 工 整 体 做 工 细 腻 豪 华 全 车 采 用 的 是 法 拉 利 玛 莎 拉 蒂 专 属 的 Ｐ ｏ ｌ ｔ ｒ ｏ ｎ ａ Ｆ ｒ ａ ｕ 豪 华 真 皮 材 质 质 感 一 流 同 时 多 处 内 饰 采 用 了 非 洲 实 木 材 质 十 分 高 档 座 椅 方 面 Ｎ ａ ｐ ｐ ａ 全 粒 面 真 皮 缝 制 乘 坐 感 舒 适 腰 部 支 撑 很 到 位 配 备 的 １ ９ 个 高 保 真 扬 声 器 的 ｈ ａ ｒ ｍ ａ ｎ ／ ｋ ａ ｒ ｄ ｏ ｎ 音 响 系 统 据 国 内 著 名 音 乐 人 亲 身 体 验 后 评 价 只 有 劳 斯 莱 斯 这 样 老 牌 贵 族 豪 车 上 的 音 效 才 能 与 之 匹 敌 ＝ 衲 辏 丛 拢 全 新 进 口 克 莱 斯 勒 ３ ０ ０ Ｃ 登 台 亮 相 后 目 标 锁 定 奥 迪 Ａ ６ Ｌ 宝 马 五 系 所 在 的 豪 华 车 市 场 而 在 市 场 预 估 的 ５ ０ 万 － ６ ０ 万 价 格 区

In [26]:
print("去除符号后的训练结果：")
train_NBayes(x_train_pro, x_test_pro, y_train, y_test)
train_TfIdf(x_train_pro, x_test_pro, y_train, y_test)
train_nGram(x_train_pro, x_test_pro, y_train, y_test,2)

去除符号后的训练结果：
朴素贝叶斯得分：0.9217
TF-IDF得分：0.9051
ngram,n为2时得分：0.9144


## Pro+: 头尾加上`<start>` `<end>` 标记

In [27]:
train_data_prop = []
for paragraph in train_data_pro:
    train_data_prop.append("<start> "+paragraph+" <end>")

print(train_data_prop[6000])


<start> 当 涂 当 涂 见 芜 湖 芜 湖 见 八 月 十 五 夜 一 似 没 柄 扇 <end>


In [28]:
x_train_prop, x_test_prop, y_train, y_test = train_test_split(train_data_prop, train_target,test_size=0.4,random_state=0)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(analyzer='char')
vector_prop = cv.fit_transform(x_train_prop)
vector_prop.todense()
vectorizer_prop = CountVectorizer(analyzer='char', max_features=5000)
vectorizer_prop.fit(x_train_prop)
print(x_train_prop[4000])

<start> 已 开 山 馆 待 抽 簪 更 要 岩 泉 欲 洗 心 常 被 松 声 迷 细 韵 急 流 花 片 落 高 岑 便 疏 浅 濑 穿 莎 径 始 有 清 光 映 竹 林 何 日 煎 茶 酝 香 酒 沙 边 同 听 暝 猿 吟 <end>


In [29]:
print("加上标记后的训练结果：")
train_NBayes(x_train_prop, x_test_prop, y_train, y_test)
train_TfIdf(x_train_prop, x_test_prop, y_train, y_test)
train_nGram(x_train_prop, x_test_prop, y_train, y_test,2)

加上标记后的训练结果：
朴素贝叶斯得分：0.9122
TF-IDF得分：0.91
ngram,n为2时得分：0.9168
